# Keras

In [40]:
import pandas as pd
import numpy as np
import seaborn as sns

from scipy.io import arff
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold

import tensorflow as tf
from tensorflow import keras

In [13]:
raw_data = arff.loadarff('./assets/genre.arff')
df = pd.DataFrame(raw_data[0])

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Columns: 241 entries, highlevel_danceability_value to genre
dtypes: float64(237), object(4)
memory usage: 1.5+ MB


,highlevel_danceability_value,highlevel_equalization_profile_value,highlevel_excitement_value,highlevel_intensity_value,highlevel_speech_music_value,highlevel_voice_instrumental_value,loudness_dynamic_complexity_dvar,loudness_dynamic_complexity_mean,loudness_dynamic_complexity_var,loudness_larm_dvar,...,tonal_chords_strength_dvar,tonal_chords_strength_mean,tonal_chords_strength_var,tonal_dissonance_dvar,tonal_dissonance_mean,tonal_dissonance_var,tonal_key_mode_value,tonal_key_strength_value,tonal_tuning_equal_tempered_deviation_value,genre
0,0.206472,0.372766,0.029087,0.093526,b'music',b'voice',-0.411385,0.024215,-0.201421,-0.532833,...,-0.255337,1.140774,-1.490395,0.494393,0.149725,-0.013634,b'minor',0.526779,-0.205793,b'blu'
1,-0.368479,0.372766,1.341977,0.093526,b'music',b'voice',-0.349921,-0.015672,-0.173045,-0.452769,...,-0.121114,0.497749,-0.901662,0.347729,-0.517640,0.184709,b'major',-0.129683,-0.667979,b'blu'
2,-0.631747,0.372766,-1.283803,-1.324366,b'speech',b'instrumental',4.235612,5.779127,3.973258,1.713613,...,-0.190639,1.408974,-0.502029,1.174496,-1.454160,2.144489,b'major',-1.022781,0.937795,b'blu'
3,-0.190116,0.372766,1.341977,0.093526,b'music',b'voice',-0.307576,0.068818,-0.216986,-0.519222,...,0.257633,0.622749,1.399033,0.791524,-0.601227,0.873673,b'major',0.074454,-0.406373,b'blu'
4,0.588278,0.372766,1.341977,0.802473,b'music',b'instrumental',-0.332032,-1.078238,0.350680,-0.765884,...,-0.309481,1.301291,-0.708195,-0.093155,0.287901,-0.644858,b'major',0.919307,-0.917709,b'blu'


In [14]:
# Split the data into numerical and categorical features
df_categorical = df.select_dtypes(include='object')
df_categorical = df_categorical.drop('genre', axis=1)
df_numerical = df.select_dtypes(exclude='object')

min_max_scaler = preprocessing.MinMaxScaler()
numerical_scaled = min_max_scaler.fit_transform(df_numerical.values)

encoder = preprocessing.OrdinalEncoder()
categorical_encoded = encoder.fit_transform(df_categorical.values)

print('Categorical features:', encoder.categories_)

print('Resulting shapes:', numerical_scaled.shape, categorical_encoded.shape)
print('Resulting types:', type(numerical_scaled), type(categorical_encoded))

Categorical features: [array([b'music', b'speech'], dtype=object), array([b'instrumental', b'voice'], dtype=object), array([b'major', b'minor'], dtype=object)]
Resulting shapes: (800, 237) (800, 3)
Resulting types: <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [15]:
X = np.hstack((numerical_scaled, categorical_encoded))

print('Features:', X.shape)
print('Features type:', type(X))

Features: (800, 240)
Features type: <class 'numpy.ndarray'>


In [16]:
le = preprocessing.LabelEncoder()
le.fit(df['genre'])
y = le.transform(df['genre'])

print("Features of the first sample: ", X[0])
print("Class of the first sample: ", y[0])

Features of the first sample:  [2.05849968e-01 6.66666571e-01 5.00000000e-01 4.28571256e-01
 1.07662232e-01 2.92794541e-01 1.98970531e-01 1.44670452e-02
 6.81261420e-01 9.46012600e-03 3.24903030e-01 1.89708482e-01
 8.13042270e-01 1.95376877e-01 1.18911965e-01 5.12198291e-01
 1.93983041e-01 1.40136526e-02 2.13441490e-01 1.51393991e-02
 4.32834508e-01 4.35091207e-03 5.00527557e-02 3.36943472e-03
 1.60258994e-02 2.13875231e-01 1.95594541e-02 5.45950607e-03
 2.03978997e-01 1.01819025e-02 1.37219577e-02 1.90879982e-01
 2.44109179e-02 2.07494017e-02 3.85624966e-01 4.97960545e-02
 1.10362700e-02 1.14581531e-01 4.79941097e-03 2.04018549e-01
 2.86317467e-01 3.40376747e-02 1.99299025e-02 1.33850006e-01
 8.35954715e-03 1.04190753e-01 3.46949918e-01 9.18480898e-02
 1.44938247e-01 5.23505317e-01 1.15454854e-01 2.78783264e-02
 1.38496771e-01 1.79918713e-02 8.58310864e-03 5.55969507e-02
 4.58629789e-03 1.62496567e-03 5.00040848e-02 1.42939903e-03
 9.73513470e-03 1.20491243e-01 5.28487318e-03 1.997004

In [35]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1)

# ..and apply normalisation
scaler = preprocessing.StandardScaler().fit(X_train_val)
norm_x_train_val = scaler.transform(X_train_val)
norm_x_test = scaler.transform(X_test)

print ("Training set: ", norm_x_train_val.shape, y_train_val.shape)
print ("Test set: ", norm_x_test.shape, y_test.shape)

Training set:  (640, 240) (640,)
Test set:  (160, 240) (160,)


In [32]:
def create_model():
    # Model structure
    model = tf.keras.Sequential([
        # keras.Input(shape=(32,240)),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(.1),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model
    

In [36]:
model = create_model()
history = model.fit(norm_x_train_val, y_train_val, epochs=10, validation_data=(norm_x_test, y_test))

Epoch 1/10
20/20 [==============================] - 1s 14ms/step - loss: 1.7983 - accuracy: 0.4047 - val_loss: 1.3115 - val_accuracy: 0.6000
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 1.0220 - accuracy: 0.6828 - val_loss: 0.9836 - val_accuracy: 0.6750
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.7073 - accuracy: 0.7953 - val_loss: 0.8511 - val_accuracy: 0.7625
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5164 - accuracy: 0.8547 - val_loss: 0.7629 - val_accuracy: 0.7750
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.3845 - accuracy: 0.9016 - val_loss: 0.7299 - val_accuracy: 0.7750
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.2801 - accuracy: 0.9328 - val_loss: 0.7233 - val_accuracy: 0.7437
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.2356 - accuracy: 0.9500 - val_loss: 0.6332 - val_accuracy: 0.7812
Epoch 8/10
20/20 [=

In [47]:
def train_evaluate(model, norm_x_train_val, y_train, norm_x_test, y_test):
    model.fit(norm_x_train_val, y_train)

kFold = StratifiedKFold(n_splits=10)
scores = np.zeros(10)
idx = 0
for train, test in kFold.split(X, y):
    model = create_model()
    scores[idx] = train_evaluate(model, X[train], y[train], X[test], y[test])
    idx += 1
print(scores)
print(scores.mean())

23/23 [==============================] - 1s 2ms/step - loss: 2.1659 - accuracy: 0.2208
[nan nan nan nan nan nan nan nan nan nan]
nan
